In [ ]:
!pip install stable_baselines3
!pip install shimmy

In [ ]:
import yfinance as yf

data = yf.download("AAPL", start="2015-01-01", end="2020-01-01")
print(data.head())


/tmp/ipython-input-3498285090.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2015-01-01", end="2020-01-01")
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2015-01-02  24.261049  24.729272  23.821673  24.718176  212818400
2015-01-05  23.577574  24.110150  23.391173  24.030263  257142000
2015-01-06  23.579798  23.839428  23.218089  23.641931  263188400
2015-01-07  23.910437  24.010294  23.677434  23.788387  160423600
2015-01-08  24.829130  24.886826  24.121248  24.238859  237458000


In [ ]:
import gym
import numpy as np
from stable_baselines3 import PPO

class StockEnv(gym.Env):
    def __init__(self, prices):
        super(StockEnv, self).__init__()
        self.prices = prices
        self.current_step = 0
        self.balance = 10000
        self.shares = 0
        self.action_space = gym.spaces.Discrete(3)  # 0=Hold,1=Buy,2=Sell
        self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape=(1,))

    def reset(self):
        self.current_step = 0
        self.balance = 10000
        self.shares = 0
        return np.array([self.prices[self.current_step]])

    def step(self, action):
        price = self.prices[self.current_step]
        if action == 1 and self.balance >= price:   # Buy
            self.shares += 1
            self.balance -= price
        elif action == 2 and self.shares > 0:       # Sell
            self.shares -= 1
            self.balance += price

        self.current_step += 1
        done = (self.current_step >= len(self.prices)-1)
        portfolio = self.balance + self.shares * self.prices[self.current_step]
        reward = portfolio - 10000  # reward = 相對於初始資產
        obs = np.array([self.prices[self.current_step]])
        return obs, reward, done, {}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
env = StockEnv(prices=data["Close"].values[:1000])
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/monitor.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.rewards.append(float(reward))
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:59: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 999      |
|    ep_rew_mean     | 9.84e+04 |
| time/              |          |
|    fps             | 1227     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 999         |
|    ep_rew_mean          | 8.48e+04    |
| time/                   |             |
|    fps                  | 932         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009871546 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | 9.12e-06    |
|    learning_rate        | 0.